# Evaluation Metrics - Regression

### Learning Objectives:
- [Regression Metrics](#Regression-Metrics)
- [Extra: ROC & AUC](#Extra:-ROC-&-AUC)

# Regression Metrics

Now that you are familiar with classification metrics, we can go over regression problems, which deal with numerical quantities rather than categorical ones.

This simple fact means that __the same evaluation metrics used in classification cannot be used for regression models__. In classification metrics, we base all metrics on binary concepts such as 'correct' and 'incorrect', then binarize multiclass problems so that the same framework applies. But this does not work with regression models. 

## Example

For instance, under a classification frame of mind, if we predict an output of 3.29742 and the true label is 3.29741, this would be classified as incorrect. What is even worse is that other predictions that are much further away would have the same categorisation. 

Therefore, with regression metrics, __we do not focus on whether something is correct or incorrect, but rather focus on how similar to the true label our prediction is.__

In [ ]:
# Loading boston dataset
X, Y = datasets.load_boston(return_X_y=True)
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(
    X, Y, test_size=0.3, random_state=0
)
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

In [ ]:
# Fitting linear regression model
myLinearModel = LinearRegression().fit(X_train, Y_train)
y_hat = myLinearModel.predict(X_test)

## Mean Squared Error (MSE), Root Mean Squared Error (RMSE) & Mean Absolute Error (MAE)

As seen previously, one metric that can be used to measure the performance of a regression model is the __mean square error (MSE)__.

We are interested in how much the prediction deviates from the label, not whether it's larger or smaller than the label, thus by squaring the errors we ensure that all terms have the same sign and the sum of any two terms can only increase our metric value. 

However, because it is measured in terms of the square of the original unit, a more commonly used evaluation metric is the __root mean square error (RMSE)__, which is the square root of the MSE. This is now measured in the same units as originally and thus becomes easier to interpret. They are given as follows for $N$ examples:

$$MSE = \frac{1}{N}\sum_{i=1}^{N}(y_{i} - \hat{y}_{i})^{2}$$
$$RMSE = \sqrt{MSE} = \sqrt{\frac{1}{N}\sum_{i=1}^{N}(y_{i} - \hat{y}_{i})^{2}}$$

## Example

Code `MSE` and `RMSE` functions taking `(predicted, targets)` arguments.

__Remember to use `MSE` when computing `RMSE`!__

In [ ]:
from sklearn.metrics import mean_squared_error


# Computing MSE and RMSE
def MSE(targets, predicted):
    return np.mean(np.square(targets - predicted))


def RMSE(targets, predicted):
    return np.sqrt(MSE(targets, predicted))


print("MSE (Python):", MSE(Y_test, y_hat))
print("MSE (scikit-learn):", mean_squared_error(Y_test, y_hat))
print()
print("RMSE (Python):", RMSE(Y_test, y_hat))
print("RMSE (scikit-learn):", mean_squared_error(Y_test, y_hat, squared=False))

Another deviation from the MSE and RMSE is known as the __mean absolute error (MAE)__, and is, simply put, the mean of the absolute value of the differences between every prediction and label.

It is the most intuitive metric out of the three because all it means is 

> on average, what is the size of our deviation from the true values

Below, we show how it is computed with both Python and scikit-learn.

## Example

Code `MAE` function

In [ ]:
from sklearn.metrics import mean_absolute_error


# Computing MAE for our Linear Model
def MAE(targets, predicted):
    return np.mean(np.abs(targets - predicted))


print("MAE (Python):", MAE(Y_test, y_hat))
print("MAE (scikit-learn):", mean_absolute_error(Y_test, y_hat))

## MAE vs MSE

So which is better: RMSE or MAE? The answer is that it depends on the context. 

- Since we square the errors when computing the RMSE, we are automatically giving a higher weight to larger errors
- MAE only takes their absolute value (so the difference is the same no matter the magnitude)

Consider the dummy example where we are handed a list of 10 errors below, where 2 of them are far larger than the rest.

In [ ]:
# Dummy example
errors = np.array([1.0, -2.5, -1.2, 253.4, 5.3, -6.9, -3.2, -100.9, 4.7, 3.3])
print("MAE: ", np.mean(np.abs(errors)))
print("RMSE: ", np.sqrt(np.mean(np.square(errors))))

From this dummy example, we can see that larger errors will make the RMSE much larger than MAE. 

In fact, the larger our errors, the larger the RMSE will be in relation to the MAE.

> While the MAE is much more intuitive than the RMSE, the RMSE is quite useful for punishing the model for larger errors. 

Nowadays, since these calculations are not done by hand, there is often no reason why we cannot use both evaluation metrics in parallel.

## R-squared (Coefficient of Determination)

The __R-squared ($\mathbf{R^{2}}$)__ is the the proportion of variance in the output that we are able to predict from the input with our model. 

This evaluation metric is important as it tackles a different aspect of performance from its MSE, RMSE and MAE counterparts. 

> We are interested in how much of the variance in the labels is explained by our model as the variation in our data is representative of the information it contains!

If our model cannot account for the variance in our data, it is not painting the whole picture!

So how can we calculate the $R^{2}$ of our model? The $R^{2}$ is a percentage of the variance from the data that is explained by our model. 
1. compute the variance of the model, which is equal to how much much our data deviates from the simplest possible model: the sample mean
2. compute the mean squared error, which tells us how much the data deviates from our model. We then use the following relationship:

$$
R^{2} = 1 - \frac{MSE}{Var(y)} = 1 - \frac{\sum_{i=1}^{N}(y - \hat{y})^{2}}{\sum_{i=1}^{N}(y - \bar{y})^{2}}
$$

Now we have a measure that will give us 0 if the model explains as much about the variability of the model as the sample mean, and 1 if it explains 100% of the variability in our data. Below we compute the $R^{2}$ using Python as well as with the in-built scikit-learn functions for our linear model.

## Example

Code `R2` function. 

__Remember to use `MSE` we came up with previously!__

In [ ]:
from sklearn.metrics import r2_score


# Computing R2
def R2(targets, predicted):
    return 1 - (MSE(targets, predicted)/np.var(targets))

print("R2 (Python):", R2(Y_test, y_hat))
print("R2 (scikit-learn):", r2_score(Y_test, y_hat))

Now, besides the $R^{2}$, the other three metrics are no longer explained in terms of a percentage. 

How can we interpret our values for MSE, RMSE and MAE? This will depend on the model and the data you are dealing with. If you deviate by 0.7 on a dataset that varies from 0-1000, this is not a large deviation. However, if your data varies from 0-1, 0.7 is a significant deviation from the truth. Therefore, these metrics are useful for comparing different models on the same data. 

> If we would like to compare the performance of two different regression models who's data are on different scales, it may be useful to normalize our evaluation metrics by the range of values in our data.